In [2]:
import cv2
import numpy as np
import os
import cvlib as cv
from cvlib.object_detection import draw_bbox

In [3]:
import ipywidgets as widgets
from IPython.display import display

# Creates sliders
minClipLength = widgets.IntSlider(
    value=5,
    min=2,
    max=20,
    step=1,
    description='Minimum clip length:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%') 
)

minComponentSize = widgets.IntSlider(
    value=100,
    min=10,
    max=500,
    step=5,
    description='Minimum component size:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%') 
)

trackingFrames = widgets.IntSlider(
    value=10,
    min=1,
    max=50,
    step=1,
    description='Number of frames to apply tracking:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%') 
)

# Displays the sliders
slider_container = widgets.VBox([minClipLength, minComponentSize, trackingFrames])
display(slider_container)

# Creates labels to display the values of the sliders
minClipLength_label = widgets.Label(value=f"minClipLength: {minClipLength.value}")
minComponentSize_label = widgets.Label(value=f"minComponentSize: {minComponentSize.value}")
trackingFrames_label = widgets.Label(value=f"trackingFrames: {trackingFrames.value}")

# Creates a container to display the labels
label_container = widgets.VBox([minClipLength_label, minComponentSize_label, trackingFrames_label])

# Displays the labels
display(label_container)

def update_labels(change):
    minClipLength_label.value = f"minClipLength: {minClipLength.value}"
    minComponentSize_label.value = f"minComponentSize: {minComponentSize.value}"
    trackingFrames_label.value = f"trackingFrames: {minComponentSize.value}"

minClipLength.observe(update_labels, names='value')
minComponentSize.observe(update_labels, names='value')
trackingFrames.observe(update_labels, names='value')


In [4]:
# Keeps only large connected components in the frame
def keepLargeComponents(I,th):
    R = np.zeros(I.shape)<0
    unique_labels = np.unique(I)
    for label in unique_labels:
        if label == 0:
            pass
        else:
            I2 = I==label
            if np.sum(I2)>th:
                R = R | I2
    return np.float32(255*R)

In [5]:
# Function to clamp bounding box coordinates
def clampBox(bbox, img_shape):
    x, y, w, h = bbox
    x = max(0, min(x, img_shape[1] - 1))
    y = max(0, min(y, img_shape[0] - 1))
    w = max(0, min(w, img_shape[1] - x))
    h = max(0, min(h, img_shape[0] - y))
    return (x, y, w, h)

In [6]:
# Saves sequence into folder in 'out'
def saveSeq(C,counter,th,trackingFrames,outputPath):
    if len(C)<th:
        return

    firstFrameOfClip = counter - len(C)

    totalDetectionFrames, totalTrackingFrames = 0, 0
    trackers = None
    consecutivePersonFrames = 1
    k = 0
    trackingPerson = False
    
    for frame in C:

        # Searches for the next frame with at least one person present
        if not trackingPerson or consecutivePersonFrames >= trackingFrames:
            # Detects objects
            bboxs,labels,conf = cv.detect_common_objects(frame, confidence= 0.4, model='yolov4-tiny')
            
            # Checks if at least one person is present
            person_indices = [i for i, label in enumerate(labels) if label == 'person']
            if person_indices:
                totalDetectionFrames += 1
                trackingPerson = True
                consecutivePersonFrames = 1 
                if len(labels) > 1:
                    # Use the indices to filter bboxs, labels, and conf
                    bboxs = [bboxs[i] for i in person_indices]
                    labels = [labels[i] for i in person_indices]
                    conf = [conf[i] for i in person_indices]
                  
                # Initializes multi-object tracker
                trackers = cv2.legacy.MultiTracker_create()
                for box in bboxs:
                    tracker = cv2.legacy.TrackerMOSSE_create()
                    box = clampBox(box, frame.shape) # Ensures the bbox is within the image

                    trackers.add(tracker, frame, box)
            else:
                trackingPerson = False
                continue # Ignores frames without people
        else:
            # Updates tracking for each object
            totalTrackingFrames += 1
            success, newBoxes = trackers.update(frame)
            bboxs = []
            labels = []  
            conf = []
            for newBox in newBoxes:
                # Convert (x, y, w, h) to (xmin, ymin, xmax, ymax)
                x, y, w, h = map(int, newBox)
                bbox = (x, y, x + int(w/2), y + int(h/2)) # Parameterize smoothing factor?
                bboxs.append(bbox)
                labels.append('person') 
                conf.append(1.0)  # Adds a placeholder confidence score
            consecutivePersonFrames += 1

        k += 1
        imName = str(firstFrameOfClip)+'_'+ str(k)+'.jpg'
        finalPath = os.path.join(outputPath, imName)
        frame = draw_bbox(frame,bboxs,labels,conf)
        cv2.imwrite(finalPath,frame)

    print(totalDetectionFrames, totalTrackingFrames, len(C))
        

In [7]:
def displaySeq(outPutPath):
    for imName in os.listdir(outPutPath):
        frame = cv2.imread(os.path.join(outPutPath,imName))
        frame = cv2.resize(frame,dsize=(600,400))
        cv2.imshow('Display',frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    cv2.destroyAllWindows()

Frame by frame capture:

In [8]:
imPath = r'src/normalFootage'

fgModel = cv2.createBackgroundSubtractorMOG2()

idx = []
C = []
counter = 0

vidName = 'Street'
outputPath = os.path.join(os.getcwd(), 'out', vidName) # add ability to name output location
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

for imName in os.listdir(imPath):
    counter += 1

    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))

    # Denoises frame
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)

    # Keeps only large connected components
    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = keepLargeComponents(labels_im,minComponentSize.value) 

    if np.sum(fgmask)>0:
        idx.append(counter)
        C.append(frame)

    # Saves clip during continous changes longer than 2 frames and at least 1 frame away from the last clip
    if len(idx) > 2 and idx[-1] > idx[-2]+1: #  or 2
        saveSeq(C[:-1],counter,minClipLength.value, trackingFrames.value, outputPath)
        idx = []
    
    # Displays original frame alongside masked frame
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask,fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2)

    # Break if 'esc' key is pressed
    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break
saveSeq(C,counter,minClipLength.value, trackingFrames.value, outputPath)
cv2.destroyAllWindows()


0 0 7
0 0 25
0 0 28
0 0 31
16 144 244


Video Capture:

In [9]:
vidPath = r'src\busyStreet.mp4'
video = cv2.VideoCapture(vidPath)

fgModel = cv2.createBackgroundSubtractorMOG2()

leastNumOfFrames = 5
idx = []
C = []
counter = 0

while(True):
    ret, frame = video.read()
    if ret == False:
        break
    counter += 1

    frame = cv2.resize(frame,dsize=(600,400))

    # Denoises frame
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)

    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = keepLargeComponents(labels_im,100) #parameterize in ui

    if np.sum(fgmask)>0:
        idx.append(counter)
        C.append(frame)

    # Saves frame during continous changes longer than 2 frames
    if len(idx) > 2 and idx[-1] > idx[-2]+1:
        saveSeq(C,counter,leastNumOfFrames,outPutPath)
        idx = []
        C = []

    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask,fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2)

    # Break if 'esc' is pressed
    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break
saveSeq(C,counter,leastNumOfFrames,outPutPath)
video.release()
cv2.destroyAllWindows()
    

NameError: name 'outPutPath' is not defined